In [23]:
#Code origin
#
#Author: Alexander Valentini  (alexander.valentini@epfl.ch)
#Made on: 26/5-2024
#Made for: Project report for course CS-552. submission on 2/6-2024
#Jessica started this file at an early stage for training, but all subsequent data analysis and filtering was done by Alexander
#This file was used for debugging done by Alexander.

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, DataCollatorWithPadding#, BitsAndBytesConfig 
from datasets import load_from_disk, load_dataset, Dataset
import numpy as np
#Using quantization, but this should probably not be in the final version:
from pathlib import Path
from typing import Dict, Optional

from trl import DPOTrainer

from peft import LoraConfig, AutoPeftModelForCausalLM


#bnb_config = BitsAndBytesConfig(
#    load_in_4bit=True,
#    llm_int8_threshold=6.0,
#    llm_int8_has_fp16_weight=False,
#    bnb_4bit_compute_dtype=torch.bfloat16,
#    bnb_4bit_use_double_quant=True,
#    bnb_4bit_quant_type="nf4",
#)

model_name = 'stabilityai/stablelm-zephyr-3b'
safetensors_path = 'sft_stablelm_zephyr_3b/25.05.2024_new_data_and_model/checkpoint-20000'

tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
model = AutoModelForCausalLM.from_pretrained(safetensors_path, attn_implementation="sdpa",
    torch_dtype=torch.bfloat16, use_safetensors = True, device_map="auto")

#model = AutoModelForCausalLM.from_pretrained(model_name, attn_implementation="sdpa",
#    torch_dtype=torch.bfloat16, use_safetensors = True, device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
model

StableLmForCausalLM(
  (model): StableLmModel(
    (embed_tokens): Embedding(50304, 2560)
    (layers): ModuleList(
      (0-31): 32 x StableLmDecoderLayer(
        (self_attn): StableLmSdpaAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (o_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
          (rotary_emb): StableLmRotaryEmbedding()
        )
        (mlp): StableLmMLP(
          (gate_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (up_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (down_proj): Linear(in_features=6912, out_features=2560, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affin

In [26]:
prompt = [{'role': 'user', 'content': 'List 3 synonyms for the word "tiny"'}]
inputs = tokenizer.apply_chat_template(
    prompt,
    add_generation_prompt=True,
    return_tensors='pt'
)

tokens = model.generate(
    inputs.to(model.device),
    max_new_tokens=20,
    temperature=0.8,
    do_sample=True
)

print(tokenizer.decode(tokens[0], skip_special_tokens=False))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<|user|>
List 3 synonyms for the word "tiny"<|endoftext|>
<|assistant|>
B. The Sun of the first of carbon has the heart of a nucleus. The earliest occurs are


In [3]:
device

device(type='cuda')

In [8]:
#Read in training and evaluation data:
import os

project_dir = os.path.dirname(os.path.abspath(os.getcwd()))
data_dir = os.path.join(project_dir, 'data/dpo_preference_data/processed/EPFL_preference_data')
preference_data = load_from_disk(data_dir)
train_dataset = preference_data['train']
eval_dataset = preference_data['eval']
#train_dataset
#print(f"Number of examples in the training set: {len(train_dataset)}")
#print(f"Number of examples in the evaluation set: {len(eval_dataset)}")

In [5]:
train_dataset[0]

{'prompt': '<|user|>\nQuestion: Let $X$ denote the random variable associated to the plaintexts and $Y$ the random variable associated to the corresponding ciphertexts. If a cryptosystem achieves perfect secrecy, then we have that \\dots?\n\nOptions:\nA. $\\Pr [X=x|Y=y] = \\Pr[Y=y]$.\nB. $\\Pr [X=x|Y=y] = \\Pr[X=x]$.\nC. $\\Pr [X=x|Y=y] = \\Pr[X=x,Y=y]$.\nD. $\\Pr [X=x] = \\Pr[Y=y]$.<|endoftext|>\n<|assistant|>\n',
 'chosen': 'The correct statement is: \\"$\\\\\\\\Pr [X=x|Y=y] = \\\\\\\\Pr[X=x]$.\\"\\n\\nPerfect secrecy in a cryptosystem means that observing the ciphertext does not give any information about the plaintext. Mathematically, this is formalized as $\\\\\\\\Pr [X=x|Y=y] = \\\\\\\\Pr[X=x]$, meaning that the probability of a certain plaintext being the true message given the ciphertext is the same as the probability of that plaintext being the true message without knowing the ciphertext.\\n\\nThe other statements are not necessarily true in the context of perfect secrecy. \\"

In [8]:
def get_stack_exchange_paired(
    data_dir: str = "data/rl",
    sanity_check: bool = False,
    cache_dir: Optional[str] = None,
    num_proc=24,
) -> Dataset:
    """Load the stack-exchange-paired dataset from Hugging Face and convert it to the necessary format.

    The dataset is converted to a dictionary with the following structure:
    {
        'prompt': List[str],
        'chosen': List[str],
        'rejected': List[str],
    }

    Prompts are structured as follows:
      "Question: " + <prompt> + "\n\nAnswer: "
    """
    dataset = load_dataset(
        "lvwerra/stack-exchange-paired",
        split="train",
        cache_dir=cache_dir,
        data_dir=data_dir,
        verification_mode="no_checks",
    )
    original_columns = dataset.column_names

    if sanity_check:
        dataset = dataset.select(range(min(len(dataset), 1000)))

    def return_prompt_and_responses(samples) -> Dict[str, str]:
        return {
            "prompt": ["Question: " + question + "\n\nAnswer: " for question in samples["question"]],
            "chosen": samples["response_j"],
            "rejected": samples["response_k"],
        }

    return dataset.map(
        return_prompt_and_responses,
        batched=True,
        num_proc=num_proc,
        remove_columns=original_columns,
    )

train_dataset = get_stack_exchange_paired(data_dir="data/rl", sanity_check=True)

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/26801833 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/42 [00:00<?, ?it/s]

Map (num_proc=24):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
train_dataset[0]

{'prompt': 'Question: I\'m writing a code using Java Swing to press the right button when I type a number key.\nBut I can\'t find what I want through search.\nThis is my code and I can\'t understand why this isn\'t working.\nPlease help me..\n\n```\nimport javax.swing.*;\nimport java.awt.Dimension;\nimport java.awt.event.*;\n\nclass class01 {\n\n    public static void main(String[] args) {\n\n        JFrame f = new JFrame("Key event test"); \n        f.setSize(230, 500);\n        f.setLayout(null);\n        f.setVisible(true);\n        f.setDefaultCloseOperation(JFrame.EXIT_ON_CLOSE);\n\n        JLabel label = new JLabel(); \n\n        JButton button1 = new JButton("Coffe"); \n        button1.setSize(100, 100);\n        button1.setLocation(0, 0);\n\n        JButton button2 = new JButton("Latte");\n        button2.setSize(100, 100);\n        button2.setLocation(0, 100);\n\n        JButton button3 = new JButton("Espresso");\n        button3.setSize(100, 100);\n        button3.setLocation

In [5]:
#Setting up Lora:
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

In [6]:
#Training arguments for DPO:

args = TrainingArguments(
    output_dir="checkpoints/dpo_model_test",               # directory to save and repository id
    num_train_epochs=1,                     # number of training epochs
    per_device_train_batch_size=12,         # batch size per device during training
    per_device_eval_batch_size=4,           # batch size for evaluation
    gradient_accumulation_steps=1,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    learning_rate=5e-5,                     # 10x higher LR than QLoRA paper
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.1,                       # warmup ratio based on QLoRA paper
    lr_scheduler_type="cosine",             # use cosine learning rate scheduler
    logging_steps=25,                       # log every 25 steps
    save_steps=400,                         # when to save checkpoint
    save_total_limit=4,                     # limit the total amount of checkpoints
    evaluation_strategy="steps",            # evaluate every 1000 steps
    eval_steps=700,                         # when to evaluate
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    push_to_hub=False,                      # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
)
 
dpo_args = {
    "beta": 0.1,                            # The beta factor in DPO loss. Higher beta means less divergence
    "loss_type": "sigmoid"                  # The loss type for DPO.
}

d:\ProgramData\Anaconda3\envs\MNLP\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
#Finding 99th percentile of data length for padding:
prompt_length = int(np.percentile([len(tokenizer(x)["input_ids"]) for x in train_dataset["prompt"]], 99))
rejected_length = int(np.percentile([len(tokenizer(x)["input_ids"]) for x in train_dataset["rejected"]], 99))
chosen_length = int(np.percentile([len(tokenizer(x)["input_ids"]) for x in train_dataset["chosen"]], 99))

print(prompt_length)
print(rejected_length)
print(chosen_length)






Token indices sequence length is longer than the specified maximum sequence length for this model (2082 > 2048). Running this sequence through the model will result in indexing errors


773
729
803


In [8]:
#This should be increased in later versions:
prompt_max_length = 900
max_seq_length = 1024

In [9]:
trainer = DPOTrainer(
    model,
    ref_model=None, # set to none since we use peft
    peft_config=peft_config,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    max_length=max_seq_length,
    max_prompt_length=prompt_max_length,
    beta=dpo_args["beta"],
    loss_type=dpo_args["loss_type"],
)

d:\ProgramData\Anaconda3\envs\MNLP\lib\site-packages\trl\trainer\dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/21390 [00:00<?, ? examples/s]

Map:   0%|          | 0/2674 [00:00<?, ? examples/s]

In [10]:
checkpoint_dir = Path(args.output_dir)
if checkpoint_dir.exists():
    checkpoints = list(checkpoint_dir.glob("checkpoint-*"))
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=lambda path: int(path.name.split("-")[-1]))
        print(f"Loading from checkpoint {latest_checkpoint}")
    else:
        latest_checkpoint = None
else:
    latest_checkpoint = None

# Continue training from the latest checkpoint if available
if latest_checkpoint:
    trainer.train(resume_from_checkpoint=str(latest_checkpoint))
else:
    trainer.train()

# save model at the end of training
trainer.save_model()

  0%|          | 0/1783 [00:00<?, ?it/s]

d:\ProgramData\Anaconda3\envs\MNLP\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
d:\ProgramData\Anaconda3\envs\MNLP\lib\site-packages\transformers\models\stablelm\modeling_stablelm.py:485: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [ ]:
peft_model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# Merge LoRA and base model and save
merged_model = peft_model.merge_and_unload()
merged_model.save_pretrained(args.output_dir, safe_serialization=True, max_shard_size="2GB")
tokenizer.save_pretrained(args.output_dir)